In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import csv

In [2]:
# TODO: Auto-parse of NCAA brackets. Not necessary now since already grabbed manually.
base_url = 'https://www.sports-reference.com'
def getTeamsUrlsForYear(year):
    postseason_url = '/cbb/postseason/' + year + '-ncaa.html'
    web = base_url + postseason_url
    html = requests.get(web).text
    soup = BeautifulSoup(html, 'html5lib')
    brackets_div = soup('div', {'id': 'bracket'})
    team_urls = {}
    for bracket in brackets_div:
        # First round of each bracket
        rounds = bracket.find('div', 'round')
        refs = rounds.findAll('a')
        for ref in refs:
            if 'schools' in ref['href']:
                team_urls[ref.text.strip()] = ref['href']

    return team_urls

In [3]:
def data_mapper(stats):
    # 'team_g', 'team_fg', 'team_fga', 'team_fg_pct', 'team_fg2', 'team_fg2a', 'team_fg2_pct',
    # 'team_fg3', 'team_fg3a', 'team_fg3_pct', 'team_ft', 'team_fta', 'team_ft_pct', 'team_orb', 'team_drb',
    # 'team_trb', 'team_ast', 'team_stl', 'team_blk', 'team_tov', 'team_pf', 'team_pts', 'opp_opp_fg',
    # 'opp_opp_fga', 'opp_opp_fg_pct', 'opp_opp_fg2', 'opp_opp_fg2a', 'opp_opp_fg2_pct', 'opp_opp_fg3',
    # 'opp_opp_fg3a', 'opp_opp_fg3_pct', 'opp_opp_ft', 'opp_opp_fta', 'opp_opp_ft_pct', 'opp_opp_orb',
    # 'opp_opp_drb', 'opp_opp_trb', 'opp_opp_ast', 'opp_opp_stl', 'opp_opp_blk', 'opp_opp_tov',
    # 'opp_opp_pf', 'opp_opp_pts'
    delim = ','
    data_string = stats['team_g'] + delim + stats['team_fg'] + delim + stats['team_fga'] + delim + \
        stats['team_fg_pct'] + delim + stats['team_fg2'] + delim + stats['team_fg2a'] + delim + \
        stats['team_fg2_pct'] + delim + stats['team_fg3'] + delim + stats['team_fg3a'] + delim + \
        stats['team_fg3_pct'] + delim + stats['team_ft'] + delim + stats['team_fta'] + delim + \
        stats['team_ft_pct'] + delim + stats['team_orb'] + delim + stats['team_drb'] + delim + \
        stats['team_trb'] + delim + stats['team_ast'] + delim + stats['team_stl'] + delim + \
        stats['team_blk'] + delim + stats['team_tov'] + delim + stats['team_pf'] + delim + \
        stats['team_pts'] + delim +  stats['opp_opp_fg'] + delim + stats['opp_opp_fga'] + delim + \
        stats['opp_opp_fg_pct'] + delim + stats['opp_opp_fg2'] + delim + stats['opp_opp_fg2a'] + delim + \
        stats['opp_opp_fg2_pct'] + delim + stats['opp_opp_fg3'] + delim + stats['opp_opp_fg3a'] + delim + \
        stats['opp_opp_fg3_pct'] + delim + stats['opp_opp_ft'] + delim + stats['opp_opp_fta'] + delim + \
        stats['opp_opp_ft_pct'] + delim + stats['opp_opp_orb'] + delim + stats['opp_opp_drb'] + delim + \
        stats['opp_opp_trb'] + delim + stats['opp_opp_ast'] + delim + stats['opp_opp_stl'] + delim +\
        stats['opp_opp_blk'] + delim + stats['opp_opp_tov'] + delim + stats['opp_opp_pf'] + delim +\
        stats['opp_opp_pts'] + '\n'
    return data_string

base_url = 'https://www.sports-reference.com'
def getTeamsData(teams_urls):
    teams_data = {}
    for team_name, team_url in teams_urls.items():
        web = base_url + team_url
        html = requests.get(web).text
        soup = BeautifulSoup(html, 'html5lib')
        table = soup.find('div', {'id': 'div_schools_totals', 'class': 'table_container'})
        thead = table.find('thead')
        tbody = table.find('tbody')
        trows = tbody.findAll('tr')
        stats = {}
        for trow in trows:
            if trow.find('th').text == "Team":
                tds = trow.findAll('td')
                prefix = 'team_'
            elif trow.find('th').text == "Opponent":
                tds = trow.findAll('td')
                prefix = 'opp_'
            else:
                continue

            for td in tds:
                stats[prefix+td['data-stat']] = td.text
        stat_string = data_mapper(stats)
        teams_data[team_name] = stat_string

    return teams_data

In [4]:
delim = ','

def createNewFileWithTeamData(in_filename, out_filename, teams_data):
    in_file = open(in_filename, 'r')
    out_file = open(out_filename, 'w')
    header = in_file.readline()
    out_file.write(header)
    for line in in_file:
        try:
            line = line.split(',')
            newline = line[0] + delim + line[1] + delim + line[2] + delim + line[3] + delim + teams_data[line[1]]
            out_file.write(newline)
        except:
            print('KEYS:', teams_data.keys())
            print(line[1])
            break

    in_file.close()
    out_file.close()

In [5]:
years = ['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2021']
for year in years:
    old_filename = './NCAA-Tournaments/' + year + '-NCAA-Tournament.xlsx'
    read_file = pd.read_excel(old_filename)
    new_filename = './NCAA-Tournaments/' + year + '-NCAA-Tournament.csv'
    read_file.to_csv(new_filename, index = None, header=True)


In [6]:
years = ['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2021']
for year in years:
    print('NOW ON YEAR: ', year)
    in_filename = './NCAA-Tournaments/' + year + '-NCAA-Tournament.csv'
    out_filename = './NCAA-Tournaments/' + year + '-NCAA-Tournament-updated.csv'
    teams_urls = getTeamsUrlsForYear(year)
    teams_data = getTeamsData(teams_urls)
    createNewFileWithTeamData(in_filename, out_filename, teams_data)
    print('----------------------------------------')

NOW ON YEAR:  1997
----------------------------------------
NOW ON YEAR:  1998
----------------------------------------
NOW ON YEAR:  1999
----------------------------------------
NOW ON YEAR:  2000
----------------------------------------
NOW ON YEAR:  2001
----------------------------------------
NOW ON YEAR:  2002
----------------------------------------
NOW ON YEAR:  2003
----------------------------------------
NOW ON YEAR:  2004
----------------------------------------
NOW ON YEAR:  2005
----------------------------------------
NOW ON YEAR:  2006
----------------------------------------
NOW ON YEAR:  2007
----------------------------------------
NOW ON YEAR:  2008
----------------------------------------
NOW ON YEAR:  2009
----------------------------------------
NOW ON YEAR:  2010
----------------------------------------
NOW ON YEAR:  2011
----------------------------------------
NOW ON YEAR:  2012
----------------------------------------
NOW ON YEAR:  2013
---------------------

In [7]:
years = ['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2021']
out_filename = './Consolidated-NCAA-Tournament-data.csv'
out_file = open(out_filename, 'w')
delim = ','
for year in years:
    print('NOW ON YEAR: ', year)
    in_filename = './NCAA-Tournaments/' + year + '-NCAA-Tournament-updated.csv'
    in_file = open(in_filename, 'r')
    if year == '1997':
        line = 'YEAR' + delim + in_file.readline()
        out_file.write(line)
    else:
        in_file.readline()
    for line in in_file: 
        out_file.write(year + delim + line)
    in_file.close()
    print('----------------------------------------')
out_file.close()

NOW ON YEAR:  1997
----------------------------------------
NOW ON YEAR:  1998
----------------------------------------
NOW ON YEAR:  1999
----------------------------------------
NOW ON YEAR:  2000
----------------------------------------
NOW ON YEAR:  2001
----------------------------------------
NOW ON YEAR:  2002
----------------------------------------
NOW ON YEAR:  2003
----------------------------------------
NOW ON YEAR:  2004
----------------------------------------
NOW ON YEAR:  2005
----------------------------------------
NOW ON YEAR:  2006
----------------------------------------
NOW ON YEAR:  2007
----------------------------------------
NOW ON YEAR:  2008
----------------------------------------
NOW ON YEAR:  2009
----------------------------------------
NOW ON YEAR:  2010
----------------------------------------
NOW ON YEAR:  2011
----------------------------------------
NOW ON YEAR:  2012
----------------------------------------
NOW ON YEAR:  2013
---------------------

In [8]:
df = pd.read_csv('./Consolidated-NCAA-Tournament-data.csv')
df

,YEAR,SEED,NAME,BRACKET,END,GAMES,TEAM_FG,TEAM_FGA,TEAM_FG%,TEAM_2P,...,OPP_FT%,OPP_ORB,OPP_DRB,OPP_TRB,OPP_AST,OPP_STL,OPP_BLK,OPP_TOV,OPP_PF,OPP_PTS
0,1997,1,UNC,East,4,35,1018,2124,0.479,830,...,0.621,447.0,680.0,1127,453.0,256.0,98.0,479.0,644.0,2334
1,1997,16,Fairfield,East,64,30,690,1609,0.429,525,...,0.680,365.0,704.0,1069,407.0,228.0,119.0,462.0,641.0,2075
2,1997,8,Indiana,East,64,33,772,1764,0.438,593,...,0.661,442.0,721.0,1163,445.0,220.0,90.0,500.0,742.0,2247
3,1997,9,Colorado,East,32,32,823,1949,0.422,676,...,0.654,498.0,825.0,1323,461.0,225.0,130.0,540.0,664.0,2228
4,1997,5,California,East,16,32,871,1905,0.457,729,...,0.700,437.0,700.0,1137,469.0,237.0,108.0,579.0,631.0,2171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,2021,14,Colgate,South,64,16,495,1005,0.493,347,...,0.685,163.0,398.0,561,178.0,88.0,47.0,191.0,282.0,1114
1532,2021,7,Florida,South,32,25,663,1407,0.471,494,...,0.709,276.0,555.0,831,316.0,189.0,83.0,357.0,444.0,1757
1533,2021,10,Virginia Tech,South,64,22,558,1240,0.450,378,...,0.697,178.0,530.0,708,236.0,128.0,73.0,266.0,382.0,1451
1534,2021,2,Ohio State,South,64,31,819,1786,0.459,567,...,0.708,307.0,734.0,1041,424.0,149.0,82.0,312.0,588.0,2204


In [9]:
df['YEAR']

0       1997
1       1997
2       1997
3       1997
4       1997
        ... 
1531    2021
1532    2021
1533    2021
1534    2021
1535    2021
Name: YEAR, Length: 1536, dtype: int64

In [14]:
df_keys_for_per_game = ['TEAM_FG', 'TEAM_FGA', 'TEAM_2P', 'TEAM_2PA', 'TEAM_3P',
    'TEAM_3PA', 'TEAM_FT', 'TEAM_FTA', 'TEAM_ORB', 'TEAM_DRB', 'TEAM_TRB', 'TEAM_AST', 'TEAM_STL', 'TEAM_BLK', 
    'TEAM_TOV','TEAM_PF', 'TEAM_PTS', 'OPP_FG', 'OPP_FGA', 'OPP_2P', 'OPP_2PA', 'OPP_3P', 'OPP_3PA', 'OPP_FT',
    'OPP_FTA', 'OPP_FT%', 'OPP_ORB', 'OPP_DRB', 'OPP_TRB', 'OPP_AST', 'OPP_STL', 'OPP_BLK', 'OPP_TOV', 
    'OPP_PF', 'OPP_PTS']
for key in df_keys_for_per_game:
    df[key] = df[key] / df['GAMES']
df

,YEAR,SEED,NAME,BRACKET,END,GAMES,TEAM_FG,TEAM_FGA,TEAM_FG%,TEAM_2P,...,OPP_FT%,OPP_ORB,OPP_DRB,OPP_TRB,OPP_AST,OPP_STL,OPP_BLK,OPP_TOV,OPP_PF,OPP_PTS
0,1997,1,UNC,East,4,35,29.085714,60.685714,0.479,23.714286,...,0.017743,12.771429,19.428571,32.200000,12.942857,7.314286,2.800000,13.685714,18.400000,66.685714
1,1997,16,Fairfield,East,64,30,23.000000,53.633333,0.429,17.500000,...,0.022667,12.166667,23.466667,35.633333,13.566667,7.600000,3.966667,15.400000,21.366667,69.166667
2,1997,8,Indiana,East,64,33,23.393939,53.454545,0.438,17.969697,...,0.020030,13.393939,21.848485,35.242424,13.484848,6.666667,2.727273,15.151515,22.484848,68.090909
3,1997,9,Colorado,East,32,32,25.718750,60.906250,0.422,21.125000,...,0.020438,15.562500,25.781250,41.343750,14.406250,7.031250,4.062500,16.875000,20.750000,69.625000
4,1997,5,California,East,16,32,27.218750,59.531250,0.457,22.781250,...,0.021875,13.656250,21.875000,35.531250,14.656250,7.406250,3.375000,18.093750,19.718750,67.843750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,2021,14,Colgate,South,64,16,30.937500,62.812500,0.493,21.687500,...,0.042813,10.187500,24.875000,35.062500,11.125000,5.500000,2.937500,11.937500,17.625000,69.625000
1532,2021,7,Florida,South,32,25,26.520000,56.280000,0.471,19.760000,...,0.028360,11.040000,22.200000,33.240000,12.640000,7.560000,3.320000,14.280000,17.760000,70.280000
1533,2021,10,Virginia Tech,South,64,22,25.363636,56.363636,0.450,17.181818,...,0.031682,8.090909,24.090909,32.181818,10.727273,5.818182,3.318182,12.090909,17.363636,65.954545
1534,2021,2,Ohio State,South,64,31,26.419355,57.612903,0.459,18.290323,...,0.022839,9.903226,23.677419,33.580645,13.677419,4.806452,2.645161,10.064516,18.967742,71.096774


In [15]:
df.to_csv('./consolidated-ncaa-tournament-per-game-data.csv', index = None, header=True)